In [ ]:
!pip -q install -U "transformers>=4.45" "accelerate>=0.33" peft bitsandbytes datasets huggingface_hub hf_xet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from huggingface_hub import login
import os
from google.colab import userdata

HF_TOKEN = userdata.get("HF_TOKEN")
if HF_TOKEN:
    login(token=HF_TOKEN)
    print("Logged in")
else:
    print("Skipping login (public access)")


In [ ]:
import torch

BASE_MODEL = userdata.get("BASE_MODEL")
ADAPTER_REPO = userdata.get("ADAPTER_REPO")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Base:", BASE_MODEL)
print("Adapter:", ADAPTER_REPO)
print("Device:", DEVICE)


In [ ]:
import os

source_path = '/content/drive/MyDrive/eece455/agents'
destination_path = '/content/agents'

if os.path.exists(source_path):
    !cp -r "{source_path}" "{destination_path}"
    print(f"Successfully copied '{source_path}' to '{destination_path}'")
else:
    print(f"Source directory '{source_path}' not found in your Google Drive. Please ensure the path is correct.")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

compute_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float16

tokenizer = AutoTokenizer.from_pretrained(ADAPTER_REPO, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype,
)

load_kwargs = dict(
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
)
load_kwargs.update({"offload_folder": "/content/offload"})

try:
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        dtype=compute_dtype,
        **load_kwargs,
    )
except TypeError:
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=compute_dtype,
        **load_kwargs,
    )

base_model.config.use_cache = True

model = PeftModel.from_pretrained(base_model, ADAPTER_REPO)
model.eval()

print("Loaded base model + LoRA adapter")


In [ ]:
from datasets import load_dataset
import random, textwrap

DATASET_ID = "flax-sentence-embeddings/stackexchange_title_best_voted_answer_jsonl"

ds = load_dataset(
    DATASET_ID,
    revision="refs/convert/parquet",
    split="train",
)

print("Full dataset size:", len(ds))
print("Keys:", ds.column_names)

ds = ds.filter(lambda x: x["site"] == "crypto")

print("Crypto dataset size:", len(ds))

def show_samples(n=3, seed=42):
    random.seed(seed)
    idxs = random.sample(range(len(ds)), n)
    for i, idx in enumerate(idxs, 1):
        q = ds[idx]["title_body"]
        a = ds[idx]["upvoted_answer"]
        print("=" * 100)
        print(f"SAMPLE {i} (idx={idx})")
        print("- Question (title+body):")
        print(textwrap.fill(q[:2000], width=110))
        print("\n- Best answer:")
        print(textwrap.fill(a[:2000], width=110))
        print()

show_samples(n=3, seed=7)


In [ ]:
def generate_answer(user_text, system_text=None, max_new_tokens=350, temperature=0.4, top_p=0.9):
    if system_text is None:
        system_text = (
            "You are a helpful assistant specialized in cryptography. "
            "Be precise with definitions, and explain reasoning clearly."
        )

    messages = [
        {"role": "system", "content": system_text},
        {"role": "user", "content": user_text},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text


In [ ]:
idx = 0
q = ds[idx]["title_body"]
print("QUESTION:\n", q[:1500], "\n")

print("MODEL OUTPUT:\n")
print(generate_answer(q))


In [ ]:
q

In [ ]:
print(generate_answer(input()))

In [ ]:
import requests
base = "127.0.0.1:8000"
for i in range(1):
    requests.post(f"{base}/api/agent/query", json={"allow_search": False, "query": ds[i]["title_body"]})
    result = response.json()
    print(result)


In [ ]:
import requests
base = "http://127.0.0.1:8000"
for i in range(1):
    requests.post(f"{base}/api/agent/query", json={"allow_search": False, "query": ds[i]["title_body"]})
    result = response.json()
    print(result)
